In [1]:
%matplotlib inline
import csv
import networkx as nx
from networkx.algorithms import bipartite
from collections import defaultdict
from mpl_toolkits.basemap import Basemap
import matplotlib.pyplot as plt

In [2]:
DATA_DIR = "./data/"
CASES_2010_2014_CSV = "CRM Cases 2010_2014 Unrestricted.csv"
CASES_2015_2019_CSV = "CRM Cases 2015_2019 Unrestricted.csv"
ECOMETRICS_VERTICAL_CSV = "CRM CBG Ecometrics Vertical.csv"
CASE_TYPES_CSV = "CRM Case Types.csv"
SHAPEFILE = "./Block+Groups+2010+BARI/Census Block Groups"

In [3]:
G = nx.Graph()

weights = defaultdict(lambda: defaultdict(int))
for filename in [CASES_2015_2019_CSV, CASES_2010_2014_CSV]:
    with open(DATA_DIR + filename, 'r') as f:
        c = csv.DictReader(f, delimiter='\t')
        count = 0
        for row in c:
            count += 1
            open_dt = row.get('OPEN_DT')
            close_dt = row.get('CLOSED_DT')
            reason = row.get('REASON')
            bg = row.get('BG_ID_10')
            if "NA" in (reason, bg):
                continue
            weights[reason][bg] += 1
            
for reason in weights:
    G.add_node(reason, bipartite='reason')
    for bg in weights[reason]:
        G.add_node(bg, bipartite='bg')
        G.add_edge(reason, bg, weight=weights[reason][bg])
        

In [4]:
def draw_bipartite(G):
    X, Y = bipartite.sets(G)
    pos = dict()
    pos.update( (n, (1, i)) for i, n in enumerate(X) ) # put nodes from X at x=1
    pos.update( (n, (2, i)) for i, n in enumerate(Y) ) 
    nx.draw(G, pos=pos)
    plt.show()


In [5]:
def setup_map(shapefile, name, scale):
    plt.figure(figsize=(12*scale,6*scale))
    m = Basemap(projection='merc', llcrnrlat=42.22, urcrnrlat=42.42,
                   llcrnrlon=-71.2, urcrnrlon=-70.98, resolution='h')
    m.readshapefile(shapefile, name, linewidth=0.7)
    m.drawcoastlines()
    m.drawstates()
    m.drawmapboundary(fill_color='aqua')
    m.fillcontinents(color='coral',lake_color='aqua')
    return m
    

#m = setup_map(SHAPEFILE, "CBGs", 2)
#try:
   # m.plot()
#except Exception as e:
    #pass



In [6]:
def export_graph(G, file, edge_filter=lambda x, y, z: True):
    with open(file, "w") as f:
        f.write("From;To;Weight\n")
        for fr, to, d in G.edges(data=True):
            if edge_filter(fr, to, d):
                f.write("{0};{1};{2}\n".format(fr, to, d['weight']))
    

In [7]:
export_graph(G, "test.txt")